## Using Large Language Models to match CV documents to job postings
See `README.md` for more background and information

### Setting up the environment
I assume you have `conda` installed (but any virtual environment with `pip` installed will do). For conda, open a terminal and type the following commands:
```bash
conda create -n job-cv-matching python=3.9
conda activate job-cv-matching
```

We create the necessary environment from the `requirements` file. In the terminal, type:
```bash
pip install -r requirements
```

In [1]:
# Then we import the packages and set some parameters
# When running this cell, make sure to select the job-cv-matching kernel from the virtual environment that we just created. If it does not show up, restart the jupyter server and try again.

from tqdm.notebook import tqdm
import json
import pandas as pd
import os
import requests
import numpy as np
import time
import pickle

base_path = 'data/'

### 1. Get the CV documents
In this example, we will use a public dataset available on [Kaggle](https://www.kaggle.com). You will need an account and an API-key to connect and download the data with the method in this notebook. You will find info on how to set this up on [this link](https://github.com/Kaggle/kaggle-api).
If you implement this on your own data, you just have to replace the call to the Kaggle-API with a call to your own source of CV documents, and then process the dataset accordingly to fit the format used below.

In [2]:
import kaggle

# Download the dataset of CV documents
!mkdir data
!kaggle datasets download leenardeshmukh/curriculum-vitae -p ./data --unzip


 72%|███████████████████████████▎          | 3.00M/4.18M [00:00<00:00, 4.68MB/s]
100%|██████████████████████████████████████| 4.18M/4.18M [00:00<00:00, 5.21MB/s]


In [9]:
# Load the data and print a few lines
base_path = 'data/'
raw = pd.read_csv(base_path+'Curriculum Vitae.csv')
raw.rename(columns={'Resume': 'cv'}, inplace=True)
raw

,Category,cv
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
11019,DotNet Developer,"Technical Skills â¢ Languages: C#, ASP .NET M..."
11020,DotNet Developer,Education Details \r\nJanuary 2014 Education ...
11021,DotNet Developer,"Technologies ASP.NET, MVC 3.0/4.0/5.0, Unit Te..."
11022,DotNet Developer,"Technical Skills CATEGORY SKILLS Language C, C..."


## 2. Summarize CV documents
CV documents are rather lengthy and often contains repeated information and are formatted as a selling text. Some of that text can act a a disturbing noise for the LLM-models that will interpret and transform the data. Since these models don't care about how nicely and well formatted the document is, we can summarize the documents to make them as information dense and to-the-point as possible. By doing so we also decrease the length, making them easier and cheaper for the GPT-based models to process.

Any suitable LLM will do here, but i have chosen GPT-based models from [openai](https://openai.com), hosted on Microsoft [Azure](https://portal.azure.com). See `README.md` pre-requisites section for more information on how to set this up.

In [10]:
import openai

# Sätt parameters for Azure openai
openai_rg_name = 'openai-lab'
openai_svc_name = 'openai-lab-rm'
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"

# Choose your openai endpoint and key that you acquire when setting up Azure openai. I have set them as environment variables by using a .env-file.
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the model for summarizing CV documents. I have used an old name, but the model is in fact gpt-35-turbo.
text_summarization_model = "text-davinci-003"